# Visualización de Red de Similitud entre Documentos

Este notebook complementa las visualizaciones de embeddings creando un gráfico de red que muestra las relaciones de similitud entre documentos.

In [ ]:
# Importamos las bibliotecas necesarias
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import networkx as nx
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from pyvis.network import Network
import matplotlib.pyplot as plt
from IPython.display import display, HTML

# Instalamos pyvis si es necesario (descomenta esta línea si no lo tienes instalado)
# !pip install pyvis networkx matplotlib

## Conexión a ChromaDB

Primero conectamos a la base de datos Chroma existente:

In [ ]:
# Configuramos el modelo de embeddings
embeddings = OllamaEmbeddings(model="mxbai-embed-large")

# Conectamos a la base de datos
db_location = "./chroma_langchain_db"
vector_store = Chroma(
    collection_name="AMVA-Reviews",
    persist_directory=db_location,
    embedding_function=embeddings
)

print(f"Conectado a la colección: {vector_store._collection.name}")

## Recuperación de Documentos y Embeddings

In [ ]:
# Obtener todos los documentos
results = vector_store.get()

# Extraer los tipos de ubicación
location_types = []
for metadata in results['metadatas']:
    location_types.append(metadata['Location-Type'])

# Obtener o generar embeddings
embeddings_data = results.get('embeddings')

# Si los embeddings no están disponibles, generarlos manualmente
if not embeddings_data:
    print("Generando embeddings manualmente...")
    embeddings_data = []
    
    for doc in results['documents']:
        emb = embeddings.embed_query(doc)
        embeddings_data.append(emb)
    
    print(f"Embeddings generados: {len(embeddings_data)}")
else:
    print(f"Embeddings recuperados directamente: {len(embeddings_data)}")

# Convertir a array numpy
embeddings_array = np.array(embeddings_data)

## Cálculo de Matriz de Similitud

In [ ]:
# Función para calcular similitud del coseno entre dos vectores
def cosine_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    return dot_product / (norm_v1 * norm_v2)

# Calcular matriz de similitud entre todos los documentos
n_docs = len(embeddings_array)
similarity_matrix = np.zeros((n_docs, n_docs))

for i in range(n_docs):
    for j in range(n_docs):
        similarity_matrix[i, j] = cosine_similarity(embeddings_array[i], embeddings_array[j])

# Creamos nombres cortos y únicos para los documentos
doc_names = [f"Doc_{i+1}" for i in range(n_docs)]
doc_labels = [doc.split()[0] for doc in results['documents']]

# Creamos un diccionario que mapea índices a documentos completos
doc_mapping = {i: {'name': doc_labels[i], 'full_text': results['documents'][i], 'type': location_types[i]} 
               for i in range(n_docs)}

# Mostrar un ejemplo del mapeo
print(f"Ejemplo de mapeo para Doc_1: {doc_mapping[0]['name']} ({doc_mapping[0]['type']})")

## Visualización de Red con NetworkX y Plotly

Vamos a crear un grafo donde:
- Cada nodo es un documento
- Las aristas conectan documentos con similitud por encima de un umbral
- El tamaño de los nodos representa la centralidad (importancia) en la red
- El color representa el tipo de ubicación

In [ ]:
# Crear un grafo basado en similitudes
similarity_threshold = 0.75  # Umbral de similitud para mostrar conexiones
G = nx.Graph()

# Agregar nodos con atributos
for i in range(n_docs):
    G.add_node(i, 
               name=doc_labels[i], 
               type=location_types[i],
               id=doc_names[i],
               full_text=results['documents'][i])

# Agregar aristas basadas en similitud
for i in range(n_docs):
    for j in range(i+1, n_docs):  # Solo la mitad superior de la matriz para evitar duplicados
        sim = similarity_matrix[i, j]
        if sim > similarity_threshold and i != j:  # Evitar autoconexiones
            G.add_edge(i, j, weight=sim)

# Calcular centralidad de grado
centrality = nx.degree_centrality(G)

# Posicionar los nodos con layout spring para mejor visualización
pos = nx.spring_layout(G, seed=42)

# Extraer coordenadas x,y del layout
node_x = [pos[i][0] for i in G.nodes()]
node_y = [pos[i][1] for i in G.nodes()]

# Mapear tipos de ubicación a colores
unique_types = list(set(location_types))
color_map = {t: i for i, t in enumerate(unique_types)}
node_colors = [color_map[G.nodes[i]['type']] for i in G.nodes()]

# Crear nodos para Plotly
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    text=[G.nodes[i]['name'] for i in G.nodes()],
    textposition="bottom center",
    hoverinfo='text',
    hovertext=[f"Documento: {G.nodes[i]['name']}\n"
               f"Tipo: {G.nodes[i]['type']}\n"
               f"Centralidad: {centrality[i]:.2f}" for i in G.nodes()],
    marker=dict(
        color=node_colors,
        colorscale='Viridis',
        colorbar=dict(title='Tipo de Ubicación'),
        showscale=True,
        size=[centrality[i] * 50 + 10 for i in G.nodes()],  # Tamaño proporcional a la centralidad
        line=dict(width=1, color='DarkSlateGrey')
    )
)

# Crear aristas para Plotly
edge_x = []
edge_y = []
edge_weights = []
edge_texts = []

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])
    
    weight = G.edges[edge]['weight']
    edge_weights.append(weight)
    
    edge_texts.append(f"{doc_labels[edge[0]]} - {doc_labels[edge[1]]}: {weight:.2f}")

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=1.5, color='#888'),
    hoverinfo='text',
    mode='lines',
    opacity=0.7
)

# Crear figura
fig = go.Figure(
    data=[edge_trace, node_trace],
    layout=go.Layout(
        title=f'Red de Similitud entre Documentos (umbral: {similarity_threshold})',
        titlefont=dict(size=16),
        showlegend=False,
        height=800,
        width=900,
        hovermode='closest',
        margin=dict(b=20,l=5,r=5,t=40),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
    )
)

fig.show()

## Visualización Interactiva con PyVis

Ahora crearemos una versión aún más interactiva usando PyVis:

In [ ]:
# Crear red interactiva con PyVis
nt = Network(height="800px", width="100%", bgcolor="#222222", font_color="white")

# Configurar opciones de física para mejorar el diseño
nt.barnes_hut()
nt.repulsion(node_distance=100, spring_length=200)

# Añadir nodos con títulos que contengan información detallada
for i in G.nodes():
    size = centrality[i] * 30 + 10  # Tamaño basado en centralidad
    # Truncar el texto completo para el título
    full_text = G.nodes[i]['full_text']
    if len(full_text) > 100:
        display_text = full_text[:100] + "..."
    else:
        display_text = full_text
        
    title = f"{G.nodes[i]['name']} ({G.nodes[i]['type']})\n{display_text}"
    
    # Mapear tipo a un color
    if G.nodes[i]['type'] == 'Municipio':
        color = '#3da4ab'
    elif G.nodes[i]['type'] == 'Corregimiento':
        color = '#f6cd61'
    elif G.nodes[i]['type'] == 'Barrio':
        color = '#fe8a71'
    else:
        color = '#424242'
        
    nt.add_node(i, label=G.nodes[i]['name'], title=title, size=size, color=color)

# Añadir aristas con info de similitud
for edge in G.edges():
    i, j = edge
    weight = G.edges[edge]['weight']
    # Ancho de línea basado en similitud
    width = weight * 5  
    title = f"Similitud: {weight:.3f}"
    nt.add_edge(i, j, title=title, width=width, value=weight)

# Añadir leyenda como HTML
legend_html = """
<div style='position:fixed; top:10px; left:20px; z-index:1; background-color:rgba(50,50,50,0.7); padding:10px; border-radius:5px'>
    <h3 style='color:white'>Leyenda</h3>
    <ul style='color:white; list-style-type:none; padding-left:10px'>
        <li><span style='display:inline-block; width:15px; height:15px; background-color:#3da4ab; margin-right:5px'></span> Municipio</li>
        <li><span style='display:inline-block; width:15px; height:15px; background-color:#f6cd61; margin-right:5px'></span> Corregimiento</li>
        <li><span style='display:inline-block; width:15px; height:15px; background-color:#fe8a71; margin-right:5px'></span> Barrio</li>
    </ul>
    <p style='color:white'>• El tamaño del nodo indica su centralidad<br>• El grosor de la línea indica similitud</p>
    <p style='color:#aaa; font-size:0.8em'>Umbral de similitud: {:.2f}</p>
</div>
""".format(similarity_threshold)

# Generar y mostrar la red
nt.set_options("""
const options = {
  "nodes": {
    "font": {
      "size": 12,
      "face": "Tahoma"
    },
    "borderWidth": 2,
    "shadow": true
  },
  "edges": {
    "color": {
      "inherit": true
    },
    "smooth": {
      "enabled": true,
      "type": "dynamic"
    },
    "shadow": true,
    "selectionWidth": 2
  },
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -80000,
      "centralGravity": 0.3,
      "springLength": 95,
      "springConstant": 0.04,
      "damping": 0.09,
      "avoidOverlap": 0.5
    },
    "maxVelocity": 50,
    "minVelocity": 0.75,
    "solver": "barnesHut",
    "timestep": 0.5,
    "adaptiveTimestep": true
  }
}
""")

# Guardar y mostrar la red
html_file = "network_similarity.html"
nt.save_graph(html_file)

# Insertar la leyenda en el archivo HTML
with open(html_file, 'r', encoding='utf-8') as f:
    html_content = f.read()
    
# Insertar la leyenda antes del cierre del body
modified_html = html_content.replace('</body>', legend_html + '</body>')

with open(html_file, 'w', encoding='utf-8') as f:
    f.write(modified_html)

# Mostrar el archivo HTML en el notebook
display(HTML(f'<a href="{html_file}" target="_blank">Abrir visualización en nueva pestaña</a>'))
display(HTML(f'<iframe src="{html_file}" width="100%" height="800px"></iframe>'))

## Análisis de Comunidades

También podemos detectar comunidades (clusters de documentos similares) usando algoritmos de detección de comunidades:

In [ ]:
# Detectar comunidades usando el algoritmo de Louvain
try:
    from community import community_louvain
    
    # Aplicar detección de comunidades
    partition = community_louvain.best_partition(G)
    
    # Mapear nodos a comunidades
    communities = {}
    for node, community_id in partition.items():
        if community_id not in communities:
            communities[community_id] = []
        communities[community_id].append(doc_labels[node])
    
    # Mostrar comunidades detectadas
    print(f"Se detectaron {len(communities)} comunidades:")
    for comm_id, nodes in communities.items():
        print(f"Comunidad {comm_id}: {', '.join(nodes)}")
    
    # Visualizar grafo con colores de comunidades
    plt.figure(figsize=(12, 8))
    pos = nx.spring_layout(G, seed=42)
    
    # Color nodes based on community
    cmap = plt.cm.rainbow
    nx.draw_networkx_nodes(G, pos, node_color=[partition[i] for i in G.nodes()], 
                           cmap=cmap, node_size=[centrality[i] * 500 + 100 for i in G.nodes()])
    nx.draw_networkx_edges(G, pos, alpha=0.5)
    nx.draw_networkx_labels(G, pos, labels={i: doc_labels[i] for i in G.nodes()}, font_size=10)
    
    plt.title("Detección de Comunidades en la Red de Documentos")
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
except ImportError:
    print("Para análisis de comunidades, instala python-louvain:")
    print("pip install python-louvain")

## Resumen de Análisis

Con estas visualizaciones de red, podemos observar:

1. **Grupos de documentos similares** - La red muestra claramente clusters de documentos semánticamente relacionados
2. **Documentos centrales** - Los nodos más grandes tienen más conexiones y son más "importantes" en la red
3. **Puentes entre grupos** - Algunos documentos conectan diferentes clusters, sirviendo como puentes semánticos
4. **Relación con tipos de ubicación** - Podemos ver si los documentos se agrupan por su tipo (Municipio, Corregimiento, Barrio)

Este tipo de análisis de red es útil para:
- Identificar temas principales en el corpus de documentos
- Descubrir relaciones no evidentes entre documentos
- Mejorar sistemas de recomendación y búsqueda